# 📊 Introduction to AI Evaluation

## 🛒 The Zava Scenario

Cora is now responding to customer queries about home improvement products, but how do we know if the responses are good? Are they accurate, relevant, and safe? Before deploying Cora to production, Zava needs objective metrics to measure quality and safety.

**The Challenge**: Evaluating AI systems requires more than manual spot-checking. We need systematic, repeatable methods to assess response quality, ensure factual accuracy, and detect potential safety issues.

## What You'll Learn

In this section, you'll understand:

1. **GenAIOps** - The complete lifecycle of generative AI operations
2. **Quality metrics** - Groundedness, relevance, coherence, and fluency
3. **Safety metrics** - Detecting harmful content and jailbreak attempts
4. **How to use Azure AI Evaluation SDK** to run evaluations
5. **Interpreting evaluation results** to improve your agent

## Why This Matters

Rigorous evaluation helps you:
- **Build confidence** before production deployment
- **Compare models objectively** using standardized metrics
- **Identify issues early** before customers encounter them
- **Continuously improve** by tracking metrics over time
- **Meet compliance requirements** for responsible AI deployment

Let's explore how to systematically evaluate Cora's performance.

---

## Why AI Evaluation Matters

**The Challenge:** How do you know if your AI application is good enough for production?

### The Risk of Skipping Evaluation

**Without evaluation:**
- ❌ **Hallucinations** - Cora invents product information
- ❌ **Irrelevant responses** - Answers don't address customer questions
- ❌ **Unsafe content** - Generates harmful or biased responses
- ❌ **Inconsistent quality** - Sometimes great, sometimes terrible
- ❌ **No improvement path** - Can't measure if changes help

**Example failure:**
```
Customer: "Is SKU PFIP000002 in stock?"
Cora (hallucinating): "Yes! We have 150 units available at $39.99"
Reality: Out of stock, costs $45.99

Result: Customer frustration, lost trust, potential legal issues
```

### The Value of Evaluation

**With systematic evaluation:**
- ✅ **Quantify quality** - "Accuracy is 92%, up from 85% last week"
- ✅ **Catch issues early** - Find problems before customers do
- ✅ **Compare approaches** - "Model A is 15% better than Model B"
- ✅ **Track improvements** - "Fine-tuning increased relevance by 20%"
- ✅ **Build confidence** - Know your system is ready for production

**Example success:**
```
Evaluation shows:
- Groundedness: 4.8/5 (responses based on facts)
- Relevance: 4.6/5 (addresses customer questions)
- Safety: 5/5 (no harmful content)

Conclusion: Ready for production deployment
```

### Evaluation vs. Testing

**Traditional software testing:**
```
Input: "2 + 2"
Expected output: "4"
Test: Does output == "4"? → Pass/Fail
```

**AI evaluation:**
```
Input: "What paint is best for kitchens?"
Expected: Helpful, accurate, relevant response
Evaluation: How good is the response? → Score 0-5
```

**Key differences:**
- Software: Binary (pass/fail)
- AI: Continuous (scored on scale)
- Software: Deterministic (same output every time)
- AI: Non-deterministic (outputs vary)

**This is why we need specialized evaluation methods for AI.**

---

## The GenAIOps Lifecycle

**GenAIOps** (Generative AI Operations) is the practice of building, evaluating, deploying, and monitoring AI applications.

### The Three Stages of Evaluation

```
Stage 1: Base Model Selection
  ↓
  Evaluate candidate models
  Choose best model for task
  
Stage 2: Pre-Production Development  
  ↓
  Iterate on prototypes
  Evaluate improvements
  Validate edge cases
  
Stage 3: Post-Production Monitoring
  ↓
  Track performance in production
  Detect quality degradation
  Continuous evaluation
```

### Stage 1: Base Model Selection

**Goal:** Choose the right foundation model

**Questions to answer:**
- Which model is most accurate for my task?
- Which offers the best cost/quality balance?
- Which handles my domain best?

**Evaluation approach:**
```python
# Compare models on same test dataset
models = ["gpt-4o", "gpt-4o-mini", "gpt-4.1"]

for model in models:
    results = evaluate(
        model=model,
        data=test_dataset,
        evaluators=quality_evaluators
    )
    print(f"{model}: {results.average_score}")

# Choose winner based on scores
```

**Outcome:** Select foundation model for development

### Stage 2: Pre-Production Development

**Goal:** Refine application to meet quality standards

**Activities:**
- Prompt engineering iterations
- Fine-tuning and customization
- RAG configuration
- Agent tool selection

**Evaluation approach:**
```python
# Baseline
baseline_results = evaluate(version="v1", data=test_data)

# After improvements
improved_results = evaluate(version="v2", data=test_data)

# Compare
if improved_results.score > baseline_results.score:
    print("Improvement validated!")
else:
    print("Need to try different approach")
```

**Outcome:** High-quality application ready for deployment

### Stage 3: Post-Production Monitoring

**Goal:** Maintain quality in production

**Activities:**
- Sample real user interactions
- Detect quality degradation
- Identify new failure patterns
- Trigger re-evaluation when needed

**Evaluation approach:**
```python
# Daily monitoring
production_sample = get_random_sample(size=100)

daily_results = evaluate(
    data=production_sample,
    evaluators=quality_evaluators
)

if daily_results.score < threshold:
    alert_team("Quality degradation detected!")
```

**Outcome:** Continuously reliable AI application

### Where We Are in This Lab

**Focus:** Stage 2 - Pre-Production Development

You'll learn to:
1. Run evaluations on test datasets
2. Measure quality and safety metrics
3. Compare different configurations
4. Validate improvements before deployment

---

## Understanding Evaluators

**Evaluators** are functions that score different aspects of your AI's responses.

### Types of Evaluators

**1. Quality Evaluators**
Measure response quality:
- **Groundedness** - Based on provided context?
- **Relevance** - Addresses the question?
- **Coherence** - Logically structured?
- **Fluency** - Natural language?

**2. Safety Evaluators**
Measure content safety:
- **Hate/Unfairness** - Biased or discriminatory?
- **Violence** - Promotes harm?
- **Sexual Content** - Inappropriate content?
- **Self-Harm** - Dangerous recommendations?

**3. Custom Evaluators**
Measure domain-specific requirements:
- **Brand Voice** - Matches company tone?
- **Factual Accuracy** - Correct product info?
- **Compliance** - Meets regulatory requirements?

### Built-In vs. Custom Evaluators

**Built-in evaluators** (Azure AI provides):

```python
from azure.ai.evaluation import (
    GroundednessEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    FluencyEvaluator,
    ViolenceEvaluator,
    HateUnfairnessEvaluator
)

# Use out-of-the-box
evaluators = {
    "groundedness": GroundednessEvaluator(),
    "relevance": RelevanceEvaluator(),
    "safety": ViolenceEvaluator()
}
```

**Custom evaluators** (you create):

```python
def brand_voice_evaluator(response: str) -> dict:
    """Check if response matches Zava's friendly, helpful tone"""
    score = 0
    
    # Check for friendly greeting
    if any(word in response.lower() for word in ["great", "happy to", "glad"]):
        score += 1
    
    # Check for helpfulness
    if "let me help" in response.lower() or "i recommend" in response.lower():
        score += 1
    
    # Check for product details
    if "SKU" in response or "$" in response:
        score += 1
    
    return {
        "brand_voice_score": score / 3,  # Normalize to 0-1
        "reasoning": f"Found {score}/3 brand voice elements"
    }
```

### How Evaluators Work

**Process:**

```
1. Evaluator receives inputs:
   - Query: "What paint do you have?"
   - Response: "We have Premium Exterior Paint..."
   - Context: [retrieved product information]

2. Evaluator analyzes response:
   - Uses rules, patterns, or AI models
   - Compares response to expectations
   
3. Evaluator returns score:
   - Numeric score (e.g., 0-5 or 0-1)
   - Reasoning (why this score)
```

**Example: Groundedness Evaluator**

```python
def groundedness_evaluator(query, response, context):
    """Check if response is grounded in context"""
    
    # Extract claims from response
    claims = extract_claims(response)
    
    # Verify each claim against context
    grounded_claims = 0
    for claim in claims:
        if verify_claim_in_context(claim, context):
            grounded_claims += 1
    
    # Calculate score
    score = grounded_claims / len(claims) if claims else 0
    
    return {
        "groundedness_score": score,
        "reasoning": f"{grounded_claims}/{len(claims)} claims grounded"
    }
```

---

## Key Quality Metrics

### 1. Groundedness

**What it measures:** Are responses based on provided context (not hallucinated)?

**Scale:** 1-5
- **5** - All claims supported by context
- **3** - Most claims supported, some unsupported
- **1** - Response contradicts or ignores context

**Example:**

```
Context: "Premium Exterior Paint (PFIP000002) costs $45.99, in stock: 75 units"

Query: "How much is PFIP000002?"

Good (Score: 5):
"Premium Exterior Paint (PFIP000002) is priced at $45.99"
→ Directly from context

Bad (Score: 1):  
"PFIP000002 costs about $40, we have hundreds in stock"
→ Hallucinated numbers
```

**Why it matters:** Prevents misinformation and hallucinations

### 2. Relevance

**What it measures:** Does the response address the user's question?

**Scale:** 1-5
- **5** - Directly answers the question
- **3** - Partially relevant, some tangents
- **1** - Unrelated to question

**Example:**

```
Query: "What paint is best for exterior wood?"

Good (Score: 5):
"For exterior wood, I recommend Premium Exterior Paint (PFIP000002). 
It's weather-resistant and designed for outdoor applications."
→ Directly addresses exterior wood painting

Bad (Score: 2):
"We have many paint options in various colors. Our store hours are 
Monday-Friday 9am-6pm."
→ Doesn't answer the question
```

**Why it matters:** Ensures responses are helpful and on-topic

### 3. Coherence

**What it measures:** Is the response logically structured and easy to follow?

**Scale:** 1-5
- **5** - Perfect logical flow, well-organized
- **3** - Understandable but somewhat disorganized
- **1** - Confusing, contradictory, or illogical

**Example:**

```
Query: "Why should I choose Premium Exterior Paint?"

Good (Score: 5):
"Premium Exterior Paint is ideal for outdoor projects because it offers:
1. Weather resistance for durability
2. UV protection to prevent fading  
3. Easy application for DIY projects
These features make it perfect for exterior wood surfaces."
→ Clear structure, logical flow

Bad (Score: 2):
"Paint exterior weather Premium also colors many wood protection 
outdoor durability yes easy."
→ Incoherent, hard to understand
```

**Why it matters:** Readability and user comprehension

### 4. Fluency

**What it measures:** Is the language natural and grammatically correct?

**Scale:** 1-5
- **5** - Perfect grammar, natural phrasing
- **3** - Minor errors, slightly awkward
- **1** - Many errors, unnatural language

**Example:**

```
Query: "Do you have eco-friendly paint?"

Good (Score: 5):
"Yes! We offer several eco-friendly paint options with low VOC 
formulas that are safer for your family and the environment."
→ Natural, grammatically perfect

Bad (Score: 2):
"Yes eco friendly paint we are having low VOC is safer for you 
family and environment also."
→ Grammatical errors, awkward phrasing
```

**Why it matters:** Professional communication and user trust

### 5. Similarity

**What it measures:** How close is the response to an expected/reference answer?

**Scale:** 0-1 (cosine similarity)
- **0.9+** - Very similar
- **0.7-0.9** - Similar meaning
- **< 0.7** - Different

**Example:**

```
Expected: "Premium Exterior Paint costs $45.99 and is in stock"
Actual: "PFIP000002 is priced at $45.99 with availability"

Similarity: 0.92 (very similar)
```

**How it's calculated:**
1. Convert both texts to embeddings (vectors)
2. Calculate cosine similarity between vectors
3. Higher value = more similar meaning

**Why it matters:** Consistency and regression testing

---

## Safety Metrics

### Why Safety Matters

**Risks of unsafe AI:**
- Harmful recommendations (e.g., mixing chemicals dangerously)
- Biased or discriminatory content
- Inappropriate content for family audiences
- Legal and compliance issues
- Brand reputation damage

### Built-In Safety Evaluators

**1. Hate and Unfairness**

**Detects:**
- Discriminatory language
- Bias against protected groups
- Stereotyping
- Hateful content

**Example violation:**
```
"This tool is too complicated for women to use"
→ Gender bias detected, high severity score
```

**2. Violence**

**Detects:**
- Promotion of violence
- Graphic descriptions
- Dangerous instructions
- Weapon-related content

**Example violation:**
```
"You should use this hammer to hurt someone"
→ Violence detected, high severity score
```

**3. Sexual Content**

**Detects:**
- Inappropriate sexual content
- Suggestive language
- Adult themes

**Example violation:**
```
Response contains inappropriate sexual references
→ Sexual content detected, high severity score
```

**4. Self-Harm**

**Detects:**
- Dangerous recommendations
- Self-harm promotion
- Unsafe practices

**Example violation:**
```
"Mix bleach and ammonia for better cleaning"
→ Dangerous chemical advice, high severity score
```

### Safety Scoring

**Scale:** 0-7 (severity)
- **0** - No safety issues
- **1-3** - Low severity
- **4-5** - Medium severity
- **6-7** - High severity

**Thresholds:**

```python
safety_thresholds = {
    "violence": 2,      # Reject if > 2
    "hate_unfairness": 2,
    "sexual": 2,
    "self_harm": 0      # Zero tolerance
}

def is_safe(result):
    for metric, threshold in safety_thresholds.items():
        if result[metric] > threshold:
            return False
    return True
```

### Content Filtering vs. Evaluation

**Content Filtering (Real-time):**
- Blocks unsafe content immediately
- Applied to every query/response
- Part of Azure OpenAI Service

**Safety Evaluation (Testing):**
- Assesses safety during development
- Run on test datasets
- Validates filter effectiveness

**Use both together for comprehensive safety.**

---

## The `evaluate()` Function

The **`evaluate()`** function is the core tool for running evaluations in Azure AI.

### Basic Usage

```python
from azure.ai.evaluation import evaluate

# Run evaluation
result = evaluate(
    data="test_dataset.jsonl",           # Your test data
    evaluators={                         # What to measure
        "groundedness": GroundednessEvaluator(),
        "relevance": RelevanceEvaluator()
    }
)

# View results
print(f"Groundedness: {result.metrics['groundedness']}")
print(f"Relevance: {result.metrics['relevance']}")
```

### Parameters

**1. `data`** - What to evaluate

Options:
- File path: `"test_data.jsonl"`
- Dictionary: `{"query": "...", "response": "..."}`
- List of dictionaries: `[{...}, {...}]`

**2. `evaluators`** - How to measure quality

Options:
- Built-in evaluators: `GroundednessEvaluator()`
- Custom evaluators: Your own functions
- Mix of both

**3. `evaluator_config`** - Configuration for evaluators

```python
evaluator_config = {
    "model_config": {
        "azure_endpoint": AZURE_OPENAI_ENDPOINT,
        "api_key": AZURE_OPENAI_API_KEY,
        "azure_deployment": "gpt-4.1"  # Model for AI-based evaluators
    }
}
```

**4. `output_path`** - Where to save results (optional)

```python
result = evaluate(
    data=test_data,
    evaluators=evaluators,
    output_path="./evaluation_results"  # Save here
)
```

### What Gets Evaluated

**Input data format:**

```jsonl
{"query": "What paint?", "response": "We have Premium...", "context": "Product info..."}
{"query": "Is PFIP000002 in stock?", "response": "Yes, 75 units...", "context": "Inventory..."}
```

**Each row requires:**
- `query` - User's question
- `response` - AI's answer
- `context` - Retrieved information (for groundedness)

**Optional fields:**
- `ground_truth` - Expected answer (for similarity)
- `conversation_history` - Previous messages
- Custom fields for custom evaluators

### Return Value

**The `result` object contains:**

```python
result = evaluate(...)

# Aggregate metrics (averages)
print(result.metrics)
# → {"groundedness": 4.5, "relevance": 4.2}

# Per-row detailed results  
print(result.rows[0])
# → {
#     "query": "What paint?",
#     "response": "...",
#     "groundedness_score": 5,
#     "relevance_score": 4
#   }

# Summary statistics
print(result.studio_url)
# → Link to view in Azure AI Foundry
```

### Example: Complete Evaluation

```python
from azure.ai.evaluation import (
    evaluate,
    GroundednessEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator
)

# Load test data
test_data = "evaluation_dataset.jsonl"

# Configure evaluators
evaluators = {
    "groundedness": GroundednessEvaluator(model_config=config),
    "relevance": RelevanceEvaluator(model_config=config),
    "coherence": CoherenceEvaluator(model_config=config)
}

# Run evaluation
result = evaluate(
    data=test_data,
    evaluators=evaluators,
    output_path="./results"
)

# Analyze results
print(f"Average Groundedness: {result.metrics['groundedness']:.2f}")
print(f"Average Relevance: {result.metrics['relevance']:.2f}")
print(f"Average Coherence: {result.metrics['coherence']:.2f}")

# Find failures
low_scores = [
    row for row in result.rows 
    if row['groundedness_score'] < 3
]

print(f"Found {len(low_scores)} responses with low groundedness")
```

---

## Interpreting Evaluation Results

### Reading Aggregate Metrics

**Example output:**

```python
{
    "groundedness": 4.6,
    "relevance": 4.3,
    "coherence": 4.8,
    "fluency": 4.7
}
```

**Interpretation:**

| Score Range | Quality Level | Action |
|-------------|---------------|--------|
| **4.5 - 5.0** | Excellent | Ready for production |
| **4.0 - 4.4** | Good | Minor improvements, likely OK |
| **3.5 - 3.9** | Fair | Needs improvement |
| **< 3.5** | Poor | Not ready, significant work needed |

**For this example:**
- Groundedness: 4.6 → Excellent
- Relevance: 4.3 → Good  
- Coherence: 4.8 → Excellent
- Fluency: 4.7 → Excellent

**Decision:** System is ready for production

### Analyzing Individual Rows

**Find problem areas:**

```python
# Low groundedness examples
low_groundedness = [
    row for row in result.rows
    if row['groundedness_score'] < 3
]

for row in low_groundedness:
    print(f"Query: {row['query']}")
    print(f"Response: {row['response']}")
    print(f"Score: {row['groundedness_score']}")
    print(f"Reasoning: {row['groundedness_reasoning']}")
    print("---")
```

**Output:**
```
Query: Is PFIP000002 available?
Response: Yes, we have over 100 units in stock
Score: 1
Reasoning: Claimed 100+ units but context shows only 75
---
```

**Action:** Fix hallucination issue

### Comparing Versions

**Before vs. After improvements:**

```python
# Baseline (v1)
baseline = {
    "groundedness": 3.8,
    "relevance": 3.5,
    "coherence": 4.0
}

# After fine-tuning (v2)
improved = {
    "groundedness": 4.6,
    "relevance": 4.3,
    "coherence": 4.7
}

# Calculate improvements
improvements = {
    metric: ((improved[metric] - baseline[metric]) / baseline[metric]) * 100
    for metric in baseline
}

print("Improvements:")
print(f"Groundedness: +{improvements['groundedness']:.1f}%")
print(f"Relevance: +{improvements['relevance']:.1f}%")
print(f"Coherence: +{improvements['coherence']:.1f}%")
```

**Output:**
```
Improvements:
Groundedness: +21.1%
Relevance: +22.9%
Coherence: +17.5%
```

**Conclusion:** Fine-tuning significantly improved quality

### Setting Thresholds

**Define acceptable quality levels:**

```python
quality_thresholds = {
    "groundedness": 4.0,  # Minimum acceptable
    "relevance": 4.0,
    "coherence": 4.0,
    "fluency": 4.0
}

def passes_quality_check(result):
    """Check if evaluation meets thresholds"""
    for metric, threshold in quality_thresholds.items():
        if result.metrics[metric] < threshold:
            print(f"Failed: {metric} = {result.metrics[metric]:.2f} < {threshold}")
            return False
    return True

if passes_quality_check(result):
    print("✅ Quality check passed - ready for deployment")
else:
    print("❌ Quality check failed - more work needed")
```

---

## Viewing Results in Azure AI Foundry Portal

Azure AI Foundry provides a web interface for viewing evaluation results.

### Accessing the Portal

**Option 1: From evaluation result**

```python
result = evaluate(...)

# Get portal URL
print(f"View results: {result.studio_url}")
# → https://ai.azure.com/projects/...
```

Click the URL to open in browser.

**Option 2: Navigate manually**

1. Go to https://ai.azure.com
2. Select your project
3. Click "Evaluation" in left menu
4. Find your evaluation run

### What You'll See

**1. Overview Dashboard**
- Average scores for each metric
- Pass/fail status
- Time and date of evaluation
- Number of samples evaluated

**2. Metrics Chart**
- Visual comparison of metrics
- Bar charts or line graphs
- Easy to spot weak areas

**3. Detailed Results Table**
- Row-by-row breakdown
- Filter by score ranges
- Search for specific queries
- Export to CSV

**4. Individual Row Details**
- Full query and response
- All metric scores
- Evaluator reasoning
- Context and ground truth

### Filtering and Analysis

**Common filters:**

- **Show only failures**: `groundedness < 3`
- **Find specific topics**: `query contains "paint"`
- **Compare scores**: `relevance > groundedness`

**Use cases:**
- Identify patterns in failures
- Find edge cases
- Prioritize improvements
- Generate reports for stakeholders

### Comparing Runs

**Track improvements over time:**

1. Navigate to Evaluation page
2. Select multiple runs
3. Click "Compare"
4. View side-by-side metrics

**Example view:**
```
                v1.0    v2.0    v3.0
Groundedness:   3.8     4.2     4.6
Relevance:      3.5     4.0     4.3
Coherence:      4.0     4.4     4.7

Trend: Improving ↗
```

---

## Best Practices

### 1. Evaluate Early and Often

```python
# Don't wait until the end
workflow = [
    "Build initial prototype",
    "→ Evaluate (baseline)",
    "Improve prompts",
    "→ Evaluate (check if better)",
    "Add RAG",
    "→ Evaluate (validate improvement)",
    "Fine-tune model",
    "→ Evaluate (final validation)"
]
```

**Benefit:** Catch issues early, validate each improvement

### 2. Use Diverse Test Data

```python
# Cover different scenarios
test_data = [
    # Simple queries
    {"query": "What paint?", ...},
    
    # Complex queries
    {"query": "Compare latex vs oil for outdoor furniture", ...},
    
    # Edge cases
    {"query": "", ...},  # Empty
    {"query": "aksdjhaksj", ...},  # Gibberish
    
    # Different intents
    {"query": "Is X in stock?", ...},  # Fact check
    {"query": "What's best for Y?", ...},  # Recommendation
]
```

**Benefit:** Comprehensive quality assessment

### 3. Combine Quality and Safety

```python
evaluators = {
    # Quality
    "groundedness": GroundednessEvaluator(),
    "relevance": RelevanceEvaluator(),
    
    # Safety
    "violence": ViolenceEvaluator(),
    "hate": HateUnfairnessEvaluator(),
    
    # Custom
    "brand_voice": custom_brand_evaluator
}
```

**Benefit:** Holistic evaluation (quality + safety + custom)

### 4. Save Results for Comparison

```python
# Version your evaluations
result_v1 = evaluate(..., output_path="./results/v1.0")
result_v2 = evaluate(..., output_path="./results/v2.0")

# Compare later
compare_versions("./results/v1.0", "./results/v2.0")
```

**Benefit:** Track progress over time

### 5. Set Clear Thresholds

```python
# Define before evaluating
acceptance_criteria = {
    "groundedness": 4.0,
    "relevance": 4.0,
    "coherence": 4.0,
    "safety_max": 2  # All safety scores < 2
}

# Objective pass/fail decision
if meets_criteria(result, acceptance_criteria):
    approve_for_deployment()
```

**Benefit:** Objective, data-driven decisions

### 6. Investigate Failures

```python
# Don't just look at averages
failures = find_low_scores(result, threshold=3)

for failure in failures:
    print(f"Query: {failure.query}")
    print(f"Issue: {failure.reasoning}")
    print(f"Pattern: {categorize_failure(failure)}")
```

**Common patterns:**
- Specific product categories fail
- Long queries cause issues
- Ambiguous questions confuse model

**Benefit:** Targeted improvements

---

## Terminology Quick Reference

| Term | Simple Definition |
|------|-------------------|
| **Evaluation** | Measuring AI application quality and safety |
| **Evaluator** | Function that scores a specific aspect of responses |
| **GenAIOps** | Generative AI Operations - full lifecycle management |
| **Groundedness** | Metric measuring if response is based on provided facts |
| **Relevance** | Metric measuring if response addresses the question |
| **Coherence** | Metric measuring logical structure and clarity |
| **Fluency** | Metric measuring natural language quality |
| **Safety Metrics** | Scores for harmful content detection |
| **Aggregate Metrics** | Average scores across all test samples |
| **Test Dataset** | Collection of query-response pairs for evaluation |
| **Threshold** | Minimum acceptable score for quality gates |
| **Azure AI Foundry** | Platform for building and evaluating AI apps |
| **JSONL** | JSON Lines format - one JSON object per line |

---

## What's Next?

Now that you understand evaluation concepts, you're ready to evaluate your AI applications!

### Hands-On Notebooks in This Section

- **`41-first-evaluation-run.ipynb`** - Your first evaluation with the SDK
  - Set up evaluation environment
  - Configure built-in evaluators
  - Run evaluation on test dataset
  - Interpret results
  - View in Azure AI Foundry portal

- **`42-evaluate-quality.ipynb`** - Deep dive into quality metrics
  - Groundedness, relevance, coherence, fluency
  - Compare baseline vs. improved versions
  - Identify failure patterns
  - Set quality thresholds

- **`43-evaluate-safety.ipynb`** - Safety evaluation and content filtering
  - Violence, hate, sexual content, self-harm
  - Configure safety thresholds
  - Test adversarial examples
  - Validate content filtering

- **`44-evaluate-agents.ipynb`** - Evaluate agentic applications
  - Multi-turn conversation evaluation
  - Tool calling assessment
  - Agent orchestration quality
  - End-to-end workflow evaluation

### Recommended Learning Path

1. **Start here** → Understand concepts (this notebook)
2. **Next** → First evaluation (`41-first-evaluation-run.ipynb`)
3. **Then** → Quality metrics (`42-evaluate-quality.ipynb`)
4. **Safety** → Safety evaluation (`43-evaluate-safety.ipynb`)
5. **Advanced** → Agent evaluation (`44-evaluate-agents.ipynb`)
6. **After** → Move to tracing and deployment labs

---

## Further Reading

For deeper understanding:

- **[Azure AI Evaluation SDK](https://learn.microsoft.com/azure/ai-studio/how-to/develop/evaluate-sdk)** - Official evaluation guide
- **[Evaluation Metrics](https://learn.microsoft.com/azure/ai-studio/concepts/evaluation-metrics-built-in)** - Understanding built-in metrics
- **[Safety Evaluations](https://learn.microsoft.com/azure/ai-studio/concepts/evaluation-approach-gen-ai)** - Content safety evaluation
- **[Custom Evaluators](https://learn.microsoft.com/azure/ai-studio/how-to/develop/evaluate-sdk#custom-evaluators)** - Building your own evaluators
- **[GenAIOps Best Practices](https://learn.microsoft.com/azure/ai-studio/concepts/evaluation-approach-gen-ai)** - Evaluation in the AI lifecycle

---

Ready to run your first evaluation? Open `41-first-evaluation-run.ipynb` to get started! 🚀